In [1]:
### lenovo desktop file path
master_file_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Data Files\\"  # Replace with the path to your directory
output_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"
combined_file_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Delivery Price Combined Files\\"

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']



In [2]:
#### Lenovo laptop file path

master_file_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\sarda\\Dropbox\\Technical Analysis(1)\\TradingView\\"  # Replace with the path to your directory
output_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"
master_file_dir = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\"


combined_file_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Delivery Price Combined Files\\"

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']

In [3]:
import os
import csv
import pandas as pd



In [4]:

def read_top_csv_file(directory, file_starts_with, num_files=1):
    # Get a list of all files in the directory
    files = [file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

    # Filter files based on the specified prefix (case-insensitive)
    matching_files = [file for file in files if file.lower().startswith(file_starts_with.lower())]

    if not matching_files:
        print(f"No files found starting with '{file_starts_with}'")
        return None

    # Sort files based on the file names in descending order
    sorted_files = sorted(matching_files, key=lambda x: x.lower(), reverse=True)

    # Get the top file with full path
    top_file_with_path = os.path.join(directory, sorted_files[0])

    # Read the CSV file into a DataFrame
    top_dataframe = pd.read_csv(top_file_with_path)

    # Return the DataFrame
    return top_file_with_path, top_dataframe

import pandas as pd

def add_nse_prefix(df, column_name):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)

    return df



In [5]:
def get_mktcap(scrip_code):
    
    if type(scrip_code) == int:
        #scrip_code = str(scrip_code)
        scrip_mktcap_df = mktcap_df[mktcap_df['BSE_Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        scrip_mktcap_df = mktcap_df[mktcap_df['NSE_Code'] == scrip_code]
        
    #scrip_mktcap_df.info()
    
    if not scrip_mktcap_df.empty:
        scrip_mktcap = scrip_mktcap_df['Market Capitalization'].values.tolist()
        
    else:
        scrip_mktcap = [0]
    
    return scrip_mktcap

    
def replace_nse_code(row):
    if pd.isna(row['NSE_Code']) or row['NSE_Code'] == "":
        return row['BSE_Code']
    else:
        return row['NSE_Code']

In [6]:
mktcap_file = master_file_dir + "MarketCap from Screener.csv"

mktcap_df = pd.read_csv(mktcap_file)
mktcap_df['BSE Code'] = mktcap_df['BSE Code'].fillna(0)
mktcap_df['BSE Code'] = mktcap_df['BSE Code'].astype(int)
mktcap_df['NSE Code'] = mktcap_df['NSE Code'].fillna("")

mktcap_df.rename(columns= { 'BSE Code' : 'BSE_Code', 
                          'NSE Code': 'NSE_Code'}, inplace =True)

mktcap_df.loc[mktcap_df['NSE_Code'] == "", "NSE_Code"] = mktcap_df['BSE_Code']


In [7]:
# Example usage:
directory_path = r'C:\Users\sarda\Dropbox\Technical Analysis (1)\TradingView'
new_high_file = "New Highs"
percent_change_file = "Tradingview"
num_top_files = 1

new_high_filename, new_high_df = read_top_csv_file(directory_path, new_high_file, num_top_files)
percent_change_filename, percent_change_df = read_top_csv_file(directory_path, percent_change_file, num_top_files)
report_date = new_high_filename[68:74]



In [8]:
tview_wl_above_500 = output_path + '_' + report_date + "_0_tview_wl_above_500" + ".csv"
tview_wl_below_500 = output_path + '_' + report_date + "_0_tview_wl_below_500" + ".csv"

new_high_wl = new_high_df[['Ticker']]
percent_change_wl = percent_change_df[['Ticker']]

new_high_nse_wl = add_nse_prefix (new_high_wl, 'Ticker')
percent_change_nse_wl = add_nse_prefix(percent_change_wl, 'Ticker')


# Concatenate the two DataFrames vertically (along rows)
tradingview_wl = pd.concat([percent_change_nse_wl, new_high_nse_wl], ignore_index=True)

## Drop duplicates based on the 'Ticker' column
tradingview_wl_unique = tradingview_wl.drop_duplicates(subset=['Ticker'])





In [9]:
# Assuming 'tradingview_wl' and 'mktcap_df' are your DataFrames

# Step 1: Create a copy of the DataFrame to avoid SettingWithCopyWarning
tradingview_wl_copy = tradingview_wl_unique.copy()

# Extract ticker symbols and remove "NSE:" prefix
tradingview_wl_copy['Ticker'] = tradingview_wl_copy['Ticker'].apply(lambda x: x.split(':')[1])

# Step 2: Merge DataFrames based on ticker symbols
tradingview_wl_copy = pd.merge(tradingview_wl_copy, mktcap_df[['NSE_Code', 'Market Capitalization']], 
                          how='left', left_on='Ticker', right_on='NSE_Code')

# Step 3: Drop unnecessary columns
tradingview_wl_copy = tradingview_wl_copy.drop(['NSE_Code'], axis=1)

# Split into two DataFrames based on market capitalization
tradingview_wl_above_500 = tradingview_wl_copy[tradingview_wl_copy['Market Capitalization'] > 500]
tradingview_wl_below_500 = tradingview_wl_copy[tradingview_wl_copy['Market Capitalization'] <= 500]

# Drop the "Market Capitalization" column from both DataFrames
tradingview_wl_above_500 = tradingview_wl_above_500.drop(['Market Capitalization'], axis=1)
tradingview_wl_below_500 = tradingview_wl_below_500.drop(['Market Capitalization'], axis=1)

# Add "NSE:" prefix to ticker symbols
tradingview_wl_above_500['Ticker'] = 'NSE:' + tradingview_wl_above_500['Ticker']
tradingview_wl_below_500['Ticker'] = 'NSE:' + tradingview_wl_below_500['Ticker']


# Save DataFrames to CSV files
tradingview_wl_above_500.to_csv(tview_wl_above_500, index=False)
tradingview_wl_below_500.to_csv(tview_wl_below_500, index=False)

print("Saved ", tview_wl_above_500, tview_wl_below_500)

# Display the resulting DataFrames
#print(tradingview_wl_unique,tradingview_wl_above_500,tradingview_wl_below_500)


Saved  C:\Users\sarda\Dropbox\SQL Stock Market Data\Python Code from ChatGPT\Output File\_231227_0_tview_wl_above_500.csv C:\Users\sarda\Dropbox\SQL Stock Market Data\Python Code from ChatGPT\Output File\_231227_0_tview_wl_below_500.csv


In [10]:
# Identify Tickers in both tradingview_wl_above_500 and tradingview_wl_below_500
tickers_above_500 = tradingview_wl_above_500['Ticker']
tickers_below_500 = tradingview_wl_below_500['Ticker']

# Find records that didn't get copied to either DataFrame
not_copied_records = tradingview_wl_copy[~tradingview_wl_copy['Ticker'].isin(tickers_above_500) & ~tradingview_wl_copy['Ticker'].isin(tickers_below_500)]

# Display the DataFrame of records that didn't get copied
#print("Records not copied to either tradingview_wl_above_500 or tradingview_wl_below_500:")
#print(not_copied_records)

# Identify records with NaN in the 'Market Capitalization' column
nan_records = not_copied_records[not_copied_records['Market Capitalization'].isna()]

# Display the DataFrame of records with NaN in the 'Market Capitalization' column
print("Records with NaN in the 'Market Capitalization' column:")
print(nan_records)


Records with NaN in the 'Market Capitalization' column:
          Ticker  Market Capitalization
12    AARTIPP.E1                    NaN
46         ARE_M                    NaN
57      S_SPOWER                    NaN
75      ASCOM.ST                    NaN
83   GOLDSTAR.ST                    NaN
94     IL_FSENGG                    NaN
98     GATECHDVR                    NaN
129   BAJAJ_AUTO                    NaN
161  AIRTELPP.E1                    NaN
169   IL_FSTRANS                    NaN
208     SURAJEST                    NaN
274     MOTISONS                    NaN
